# Data cleaning and classification

In [13]:
# Package import and data directory
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt
import numpy as np
import qiime2 as q2
import seaborn as sns

%matplotlib inline

data_dir = 'data'
data_dir_pre = 'data/preprocessing'

##  Get data

In [4]:
# Data and metadata import
if not os.path.exists('data/pundemic_metadata.tsv'):
    ! wget -O $data_dir/pundemic_metadata.tsv https://polybox.ethz.ch/index.php/s/7LxWSbaw2q37yof/download

if not os.path.exists('data/pundemic_forward_reads.qza'):
    ! wget -O $data_dir/pundemic_forward_reads.qza https://polybox.ethz.ch/index.php/s/o8HqHJqvuf9e2on/download


In [4]:
meta_df = pd.read_csv('data/pundemic_metadata.tsv', sep='\t', index_col=0)

## Metadata df overview

In [6]:
meta_df.head()

,patient_id,age,sex,ethnicity,continent,country,region,city,group,disease_subgroup,blinded_clinical_response,puns_per_hour_pre_treatment,puns_per_hour_post_treatment,time_point
id,,,,,,,,,,,,,,
SRR10505051,1048,36,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,Placebo,NR,9.0,8.0,post-treatment
SRR10505052,1048,36,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,Placebo,NR,9.0,8.0,pre-treatment
SRR10505053,1045,29,male,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,Placebo,Res,6.0,0.0,pre-treatment
SRR10505054,1045,29,male,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,Placebo,Res,6.0,0.0,post-treatment
SRR10505055,1044,34,male,Indian Subcontinental,Europe,Switzerland,Zurich,Zurich,Puns,Placebo,NaN,4.0,NaN,pre-treatment


In [6]:
# We have 3 groups, placebo FMT and donor
meta_df['disease_subgroup'].unique() 

array(['Placebo', 'FMT', 'donor'], dtype=object)

In [7]:
# Creating DFs of our different populations for potential later use
patient_df = meta_df[(meta_df['disease_subgroup'] == "placebo") | (meta_df['disease_subgroup'] == "FMT")]
patient_df.to_csv(f'{data_dir}/patient_df', sep = '\t')

In [8]:
donor_df = meta_df[(meta_df['disease_subgroup'] == "donor")]
donor_df.to_csv(f'{data_dir}/donor_df', sep = '\t')

In [9]:
meta_df[(meta_df['time_point'] == "pre-treatment") | (meta_df['time_point'] == "post-treatment")]

,patient_id,age,sex,ethnicity,continent,country,region,city,group,disease_subgroup,blinded_clinical_response,puns_per_hour_pre_treatment,puns_per_hour_post_treatment,time_point
id,,,,,,,,,,,,,,
SRR10505051,1048,36,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,Placebo,NR,9.0,8.0,post-treatment
SRR10505052,1048,36,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,Placebo,NR,9.0,8.0,pre-treatment
SRR10505053,1045,29,male,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,Placebo,Res,6.0,0.0,pre-treatment
SRR10505054,1045,29,male,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,Placebo,Res,6.0,0.0,post-treatment
SRR10505055,1044,34,male,Indian Subcontinental,Europe,Switzerland,Zurich,Zurich,Puns,Placebo,NaN,4.0,NaN,pre-treatment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR10505141,1001,57,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,FMT,Res,7.0,2.0,pre-treatment
SRR10505142,1001,57,female,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,FMT,Res,7.0,2.0,post-treatment
SRR10505153,2225,34,male,Caucasian,Europe,Switzerland,Zurich,Zurich,Puns,FMT,NR,6.0,5.0,pre-treatment


From initial exploration, there are 105 samples from different groups:

- Donor of the FMT treatment group
- Placebo treatment
- FMT treatment

And we also have different timepoints of data colleciton, pre and post treatment.

# Sequence Import

In [20]:
! qiime tools peek $data_dir/pundemic_forward_reads.qza

UUID:        15b104e4-bd22-4469-92ac-e08f76767733
Type:        SampleData[SequencesWithQuality]
Data format: SingleLanePerSampleSingleEndFastqDirFmt


In [5]:
! qiime demux summarize \
    --i-data $data_dir/pundemic_forward_reads.qza \
    --o-visualization $data_dir_pre/pundemic_forward_reads.qzv

Saved Visualization to: data/preprocessing/pundemic_forward_reads.qzv


In [2]:
Visualization.load(f"{data_dir_pre}/pundemic_forward_reads.qzv")

<visualization: Visualization uuid: ba187720-d548-42ca-ba4e-f34bd49e3b09>

From this summary, we find that our samples have around a 10 fold spread in terms of sample reads, with the median around 44,000. We have been informed by the TAs there were primers still in this dataset, which we remove in the following cells:

# Cut primer sequences in data

In [4]:
# Quality filtering
# no cutoff to keep longer reads for our ITS data
! qiime quality-filter q-score \
    --i-demux $data_dir/pundemic_forward_reads.qza \
    --p-min-quality 15 \
    --p-min-length-fraction 0 \
    --p-quality-window 3 \
    --o-filtered-sequences $data_dir_pre/pundemic_forward_reads_QCfiltered.qza \
    --o-filter-stats $data_dir_pre/pundemic_forward_reads_QCfiltered_stats.qza


^C

Aborted!


In [8]:
! qiime demux summarize \
    --i-data $data_dir_pre/pundemic_forward_reads_QCfiltered.qza  \
    --o-visualization $data_dir_pre/pundemic_forward_reads_QCfiltered.qzv

Saved Visualization to: data/preprocessing/pundemic_forward_reads_QCfiltered.qzv


In [12]:
Visualization.load(f"{data_dir_pre}/pundemic_forward_reads_QCfiltered.qzv")

<visualization: Visualization uuid: 2f8aefca-ac27-48b1-8aa9-f1edfea53dfd>

In [14]:
# Trim forward primer

! qiime cutadapt trim-single \
  --i-demultiplexed-sequences $data_dir_pre/pundemic_forward_reads_QCfiltered.qza \
  --p-front CTTGGTCATTTAGAGGAAGTAA \
  --p-overlap 8 \
  --p-error-rate 0.2 \
  --p-discard-untrimmed \
  --verbose \
  --o-trimmed-sequences $data_dir_pre/pundemic_forward_reads_trimmed_QCfiltered.qza | tee $data_dir_pre/cutadapt_results.txt 


This is cutadapt 4.9 with Python 3.9.19
Command line parameters: --cores 1 --error-rate 0.2 --times 1 --overlap 8 --minimum-length 1 -q 0,0 --quality-base 33 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-3yze1esv/SRR10505051_0_L001_R1_001.fastq.gz --front CTTGGTCATTTAGAGGAAGTAA --discard-untrimmed /tmp/qiime2/jovyan/data/f11bd813-cc41-4c3f-be1f-86834fa49ff0/data/SRR10505051_0_L001_R1_001.fastq.gz
Processing single-end reads on 1 core ...
Done           00:00:00        37,345 reads @  15.1 µs/read;   3.97 M reads/minute
Finished in 0.567 s (15.183 µs/read; 3.95 M reads/minute).

=== Summary ===

Total reads processed:                  37,345
Reads with adapters:                    37,290 (99.9%)

== Read fate breakdown ==
Reads that were too short:                   0 (0.0%)
Reads written (passing filters):        37,290 (99.9%)

Total basepairs processed:     7,533,419 bp
Quality-trimmed:                       0 bp (0.0%)
Total written (filtered):      6,717,839 bp (89.2%)

=== Ad

In [11]:
! qiime demux summarize \
    --i-data $data_dir_pre/pundemic_forward_reads_trimmed_QCfiltered.qza  \
    --o-visualization $data_dir_pre/pundemic_forward_reads_trimmed_QCfiltered.qzv

Saved Visualization to: data/preprocessing/pundemic_forward_reads_trimmed_QCfiltered.qzv


In [11]:
Visualization.load(f"{data_dir_pre}/pundemic_forward_reads_trimmed_QCfiltered.qzv")

<visualization: Visualization uuid: b72c4dab-097f-4f31-b4f9-1c983df50757>

In [16]:
# Trim reverse compliment of back primer 3' 
! qiime cutadapt trim-single \
  --i-demultiplexed-sequences $data_dir_pre/pundemic_forward_reads_trimmed_QCfiltered.qza \
  --p-adapter GCATCGATGAAGAACGCAGC \
  --p-error-rate 0.2 \
  --p-overlap 16 \
  --verbose \
  --o-trimmed-sequences $data_dir_pre/pundemic_forward_reads_trimmed_QCfiltered_2.qza | tee $data_dir_pre/cutadapt_forward_backward_results.txt


This is cutadapt 4.9 with Python 3.9.19
Command line parameters: --cores 1 --error-rate 0.2 --times 1 --overlap 16 --minimum-length 1 -q 0,0 --quality-base 33 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-1s675qsz/SRR10505051_0_L001_R1_001.fastq.gz --adapter GCATCGATGAAGAACGCAGC /tmp/qiime2/jovyan/data/7740ea85-78da-4ff1-8fed-00f35a854b32/data/SRR10505051_0_L001_R1_001.fastq.gz
Processing single-end reads on 1 core ...
Done           00:00:00        22,851 reads @  17.8 µs/read;   3.38 M reads/minute
Finished in 0.408 s (17.851 µs/read; 3.36 M reads/minute).

=== Summary ===

Total reads processed:                  22,851
Reads with adapters:                    13,152 (57.6%)

== Read fate breakdown ==
Reads that were too short:                   2 (0.0%)
Reads written (passing filters):        22,849 (100.0%)

Total basepairs processed:     5,131,551 bp
Quality-trimmed:                       0 bp (0.0%)
Total written (filtered):      4,915,374 bp (95.8%)

=== Adapter 1 ===

Seque

In [17]:
! qiime demux summarize \
    --i-data $data_dir_pre/pundemic_forward_reads_trimmed_QCfiltered_2.qza  \
    --o-visualization $data_dir_pre/pundemic_forward_reads_trimmed_QCfiltered_2.qzv

Saved Visualization to: data/preprocessing/pundemic_forward_reads_trimmed_QCfiltered_2.qzv


In [18]:
Visualization.load(f"{data_dir_pre}/pundemic_forward_reads_trimmed_QCfiltered_2.qzv")

<visualization: Visualization uuid: ed354d00-ba42-45e5-9e6a-8736222185e1>

Through this process of filtering we are left with:

7,977,313 starting reads

7,792,119 after primer trimming and QC filtering

# Denoising
Our sequences originate from the ITS-1 region and were sequenced on an Illumina MiSeq machine. The sequence files contain demultiplexed single-end sequences.

We're using DADA2. It removes chimeras from our demuxed data. The output of applying DADA gives us features which are representative sequences for organisms.

### Denoising with trimmed and quality filtered reads

In [ ]:
# Dada2 denoising using truncation length

! qiime dada2 denoise-single \
    --i-demultiplexed-seqs $data_dir_pre/pundemic_forward_reads_trimmed_QCfiltered_2.qza \
    --p-trunc-len 0 \
    --o-table $data_dir/dada/dada2_table_18-11.qza \
    --o-representative-sequences $data_dir/dada/dada2_rep_seq18-11.qza \
    --o-denoising-stats $data_dir/dada/dada2_stats18-11.qza

In [ ]:
! qiime metadata tabulate \
    --m-input-file $data_dir/dada/dada2_stats18-11.qza \
    --o-visualization $data_dir/dada/dada2_stats_18-11.qzv

In [4]:
Visualization.load(f"{data_dir}/dada/dada2_stats_18-11.qzv")

<visualization: Visualization uuid: 9a9827c6-684f-458c-a131-921e5f171f3c>

In [ ]:
! qiime feature-table tabulate-seqs \
    --i-data $data_dir/dada/dada2_rep_seq18-11.qza \
    --o-visualization $data_dir/dada/dada2_rep_seq_18-11.qzv

In [5]:
Visualization.load(f"{data_dir}/dada/dada2_rep_seq_18-11.qzv")

<visualization: Visualization uuid: c42eec85-321c-4ce0-96a8-cdc830fb5366>

We can also visualize the feature table. We will include the metadata file to enrich the table with information on sample groups.

In [ ]:
! qiime feature-table summarize \
    --i-table $data_dir/dada/dada2_table_18-11.qza \
    --m-sample-metadata-file $data_dir/pundemic_metadata.tsv \
    --o-visualization $data_dir/dada/dada2_table_18-11.qzv

In [6]:
Visualization.load(f"{data_dir}/dada/dada2_table_18-11.qzv")

<visualization: Visualization uuid: f31ed705-ed4b-4e54-8021-792a79372dd0>

After denoising, we are left with 7,113,576 of the 7,792,119 trimmed and QC filtered reads.

# Classification - training a classifier

In [ ]:
# Download the UNITE v10.0 reference data for “ALL EUKARYOTES” from UNITE using RESCRIPt
! qiime rescript get-unite-data \
    --p-version '10.0' \
    --p-taxon-group 'eukaryotes' \
    --p-cluster-id '99' \
    --p-singletons False \
    --o-taxonomy $data_dir/unite/unite-v10-99-all-euk-taxonomy.qza \
    --o-sequences $data_dir/unite/unite-v10-99-all-euk-sequences.qza \
    --verbose

In [ ]:
# Remove the species hypotheses information
! qiime rescript edit-taxonomy \
    --i-taxonomy $data_dir/unite/unite-v10-99-all-euk-taxonomy.qza \
    --o-edited-taxonomy $data_dir/unite/unite-v10-99-all-euk-taxonomy-no-sh.qza \
    --p-search-strings ';sh__.*' \
    --p-replacement-strings '' \
    --p-use-regex

In [ ]:
# Train NB classifier
! qiime feature-classifier fit-classifier-naive-bayes \
    --i-reference-reads unite-v10-99-all-euk-sequences.qza \
    --i-reference-taxonomy unite-v10-99-all-euk-taxonomy-no-sh.qza \
    --o-classifier unite-v10-99-all-euk-classifier.qza

## have to do this on euler ^^
## upload to euler
scp unite-v10-99-all-euk-sequences.qza ameara@euler.ethz.ch:/cluster/scratch/ameara/apbio2/unite/
scp unite-v10-99-all-euk-taxonomy-no-sh.qza ameara@euler.ethz.ch:/cluster/scratch/ameara/apbio2/unite/
scp dada2_table_18-11.qza ameara@euler.ethz.ch:/cluster/scratch/ameara/apbio2/unite/

sbatch --time=360 --ntasks=1 --mem-per-cpu=128G scripts/train.sh 

# script for training
#!/bin/bash
source ~/miniconda3/etc/profile.d/conda.sh
conda activate q2
DATA="/cluster/scratch/ameara/apbio2/data"
CDATA="/cluster/scratch/ameara/apbio2/unite"

echo "Start training!"

qiime feature-classifier fit-classifier-naive-bayes \
    --i-reference-reads $CDATA/unite-v10-99-all-euk-sequences.qza \
    --i-reference-taxonomy $CDATA/unite-v10-99-all-euk-taxonomy-no-sh.qza \
    --o-classifier $CDATA/unite-v10-99-all-euk-classifier.qza

echo "training done."

In [ ]:
# Retrieve data from euler
scp ameara@euler.ethz.ch:/cluster/scratch/ameara/apbio2/data/taxonomy_trained_18-11.qza .

In [ ]:
# Plot the results
# for data from pretrained classifier
! qiime taxa barplot \
    --i-table $data_dir/dada/dada2_table_18-11.qza \
    --i-taxonomy $data_dir/taxonomy_classification/taxonomy_trained_18-11.qza \
    --m-metadata-file $data_dir/pundemic_metadata.tsv \
    --o-visualization $data_dir/taxonomy_classification/taxa-barplot_trained_18-11.qzv

In [7]:
Visualization.load(f"{data_dir}/taxonomy_classification/taxa-barplot_trained_18-11.qzv")

<visualization: Visualization uuid: 4bde82b7-1633-4b45-83ed-dd65c60ce6e9>

This was not used in our final analysis, as Blast classification performed better


# Closed reference clustering

In [ ]:
# Close-reference clustering done on Euler
! qiime vsearch cluster-features-closed-reference \
  --i-table $data_dir/dada/dada2_table_18-11.qza \
  --i-sequences $data_dir/dada/dada2_rep_seq18-11.qza \
  --i-reference-sequences $data_dir/unite_db/unite-v10-99-all-euk-sequences.qza \
  --p-perc-identity 0.90 \
  --o-clustered-table $data_dir/closed_reference_cluster/cr90_feature_table.qza \
  --o-clustered-sequences $data_dir/closed_reference_cluster/cr90_rep_seqs.qza \
  --o-unmatched-sequences $data_dir/closed_reference_cluster/cr90_unmatched_rep_seqs.qza \
  --p-threads 0

# sbatch --time=360 --ntasks=1 --mem-per-cpu=128G scripts/cluster_closed_cr90_18-11.sh

In [ ]:
! qiime feature-table tabulate-seqs \
    --i-data $data_dir/closed_reference_cluster/cr90_rep_seqs.qza \
    --o-visualization $data_dir/closed_reference_cluster/cr90_rep_seqs.qzv

! qiime feature-table summarize \
    --i-table $data_dir/closed_reference_cluster/cr90_feature_table.qza \
    --m-sample-metadata-file $data_dir/pundemic_metadata.tsv \
    --o-visualization $data_dir/closed_reference_cluster/cr90_feature_table.qzv

In [8]:
Visualization.load(f"{data_dir}/closed_reference_cluster/cr90_rep_seqs.qzv")

<visualization: Visualization uuid: 53b42a21-1ad4-4721-9d29-36ac4f15990a>

In [9]:
Visualization.load(f"{data_dir}/closed_reference_cluster/cr90_feature_table.qzv")

<visualization: Visualization uuid: a578ea0d-172f-4db1-ae7a-9a60977962ce>

In [ ]:
# Start classifying via blast

! qiime feature-classifier classify-consensus-blast \
    --i-query $data_dir/closed_reference_cluster/cr90_rep_seqs.qza \
    --i-reference-taxonomy $data_dir/unite_db/unite-v10-99-all-euk-taxonomy.qza \
    --i-reference-reads $data_dir/unite_db/unite-v10-99-all-euk-sequences.qza \
    --p-perc-identity 0.9 \
    --p-query-cov 0.7 \
    --p-num-threads 0 \
    --o-classification $data_dir/taxonomy_classification/taxonomy_blast_18-11.qza  \
    --o-search-results $data_dir/taxonomy_classification/top_hits_blast_18-11.qza 


In [ ]:
# Plot the results
# for data from blast classification from clustering
! qiime taxa barplot \
    --i-table $data_dir/closed_reference_cluster/cr90_feature_table.qza \
    --i-taxonomy $data_dir/taxonomy_classification/taxonomy_blast_18-11.qza \
    --m-metadata-file $data_dir/pundemic_metadata.tsv \
    --o-visualization $data_dir/taxonomy_classification/taxa-barplot_18-11_blast.qzv

In [ ]:
Visualization.load(f"{data_dir}/taxonomy_classification/taxa-barplot_18-11_blast.qzv")

In [ ]:
# Filter out non fungal data
! qiime taxa filter-table \
    --i-table $data_dir/closed_reference_cluster/cr90_feature_table.qza \
    --i-taxonomy $data_dir/taxonomy_classification/taxonomy_blast_18-11.qza \
    --p-exclude k__Viridiplantae,k__Stramenopila,k__Metazoa\
    --p-include c__ \
    --o-filtered-table $data_dir/closed_reference_cluster/table-filtered.qza

! qiime taxa filter-seqs \
    --i-sequences $data_dir/closed_reference_cluster/cr90_rep_seqs.qza \
    --i-taxonomy $data_dir/taxonomy_classification/taxonomy_blast_18-11.qza \
    --p-exclude k__Viridiplantae,k__Stramenopila,k__Metazoa\
    --p-include c__ \
    --o-filtered-sequences $data_dir/closed_reference_cluster/rep-seqs-filtered.qza

In [ ]:
! qiime feature-table tabulate-seqs \
    --i-data $data_dir/closed_reference_cluster/rep-seqs-filtered.qza \
    --o-visualization $data_dir/closed_reference_cluster/rep_seqs_filtered.qzv

! qiime feature-table summarize \
    --i-table $data_dir/closed_reference_cluster/table-filtered.qza \
    --m-sample-metadata-file $data_dir/pundemic_metadata.tsv \
    --o-visualization $data_dir/closed_reference_cluster/table-filtered.qzv

In [ ]:
Visualization.load(f"{data_dir}/closed_reference_cluster/rep_seqs_filtered.qzv")

In [ ]:
Visualization.load(f"{data_dir}/closed_reference_cluster/table-filtered.qzv")

In [ ]:
# Plot the results
# for data from blast classification from clustering
! qiime taxa barplot \
    --i-table $data_dir/closed_reference_cluster/table-filtered.qza \
    --i-taxonomy $data_dir/taxonomy_classification/taxonomy_blast_18-11.qza \
    --m-metadata-file $data_dir/pundemic_metadata.tsv \
    --o-visualization $data_dir/taxonomy_classification/taxa-barplot_blast_filtered.qzv

In [3]:
Visualization.load(f"{data_dir}/taxonomy_classification/taxa-barplot_blast_filtered.qzv")

<visualization: Visualization uuid: b4159564-32ef-4653-a735-90d5c625fa77>

# Classifying with pretrained

Attempted using a classifier pre trained on the Unite DB for our data

In [ ]:
# Retrieve classification from pretrained from unite_ver10_99_all
scp ameara@euler.ethz.ch:/cluster/scratch/ameara/apbio2/data/taxonomy_18-11.qza .

In [ ]:
# Plot the results
# for data from pretrained classifier
! qiime taxa barplot \
    --i-table $data_dir/dada/dada2_table_18-11.qza \
    --i-taxonomy $data_dir/taxonomy_classification/taxonomy_pretrained_18-11.qza \
    --m-metadata-file $data_dir/pundemic_metadata.tsv \
    --o-visualization $data_dir/taxonomy_classification/taxa-barplot_18-11_pretrained.qzv

In [ ]:
Visualization.load(f"{data_dir}/taxonomy_classification/taxa-barplot_18-11_pretrained.qzv")

This was not used in our final analysis, as Blast classification performed better
